In [1]:
dbfile = '/home/rafa/Documentos/outlook/emails2.db'

In [2]:
from zipfile import ZipFile
import logging
import sys
import argparse
from sqlalchemy import create_engine
import sqlalchemy.orm as orm

In [3]:
import attr
from lib import Email, Base

In [4]:
engine = create_engine(f'sqlite:///{dbfile}', echo=False)
Base.metadata.create_all(engine)

In [5]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [6]:
session = Session()
emails = session.query(Email).count()
emails

32639

In [7]:
first =session.query(Email).filter(Email.id == 1)
first

In [8]:
item = first.one()

In [9]:
# Esto es necesario para que los mensajes se guarden
# en UTF-8, y no en base64.
from email import charset
charset.add_charset('utf-8', charset.SHORTEST)

In [10]:
import mailbox
mdir = mailbox.Maildir("/home/rafa/Documentos/outlook/mail", None, True)

#for item in session.query(Email.label).distinct():
#    mdir.add_folder(item.label.replace('/','.'))


In [11]:
maxid = -1
boxes = dict()
total = 1
accum = 0
while total > 0:
    total = 0
    mails = session.query(Email).filter(Email.id>maxid).order_by(Email.id).limit(100)
    for msg in mails:
        folder = msg.label.replace("/", ".")
        box = boxes.get(folder, None)
        if box is None:
            box = mdir.add_folder(folder)
            boxes[folder] = box
        box.add(msg.message())
        maxid = msg.id
        total += 1
    accum += total
    print("Saved %d messages" % accum)


SELECT message.id AS message_id, message.path AS message_path, message.account AS message_account, message.label AS message_label, message."isOutgoingMeetingResponse" AS "message_isOutgoingMeetingResponse", message."isOutgoing" AS "message_isOutgoing", message."isCalendarMessage" AS "message_isCalendarMessage", message."hasHTML" AS "message_hasHTML", message."hasRichText" AS "message_hasRichText", message."sentTime" AS "message_sentTime", message."receivedTime" AS "message_receivedTime", message."displayTo" AS "message_displayTo", message.subject AS message_subject, message.preview AS message_preview, message.body AS message_body 
FROM message 
WHERE message.id > ? ORDER BY message.id
 LIMIT ? OFFSET ?
Saved 100 messages
SELECT message.id AS message_id, message.path AS message_path, message.account AS message_account, message.label AS message_label, message."isOutgoingMeetingResponse" AS "message_isOutgoingMeetingResponse", message."isOutgoing" AS "message_isOutgoing", message."isCalen